In [16]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

import pandas as pd

import plotly as py
import plotly.graph_objs as go
py.tools.set_credentials_file(username='pravschotu', api_key='uhXw43S1XuGspEyvOv4f')

In [2]:
#engine = create_engine('sqlite:///./FPA_FOD_20170508.sqlite')
#Base.metadata.create_all(engine)

In [3]:
db_uri = 'sqlite:///./data/FPA_FOD_20170508.sqlite'
engine = create_engine(db_uri)

inspector = inspect(engine)

# Get table information
#print(inspector.get_table_names())

# Get column information
#print(inspector.get_columns('Fires'))

In [4]:
#test = engine.execute('SELECT fire_year, county, fips_code, count(*) FROM Fires WHERE state == "CA" GROUP BY fire_year, county, fips_code')
#for i in test:
#    print(i)

In [5]:
# Store data in dataframe
df_us_fires = pd.read_sql('SELECT FIRE_NAME, FIRE_YEAR, FIRE_SIZE, STAT_CAUSE_DESCR, LONGITUDE, LATITUDE, FIPS_CODE, FIPS_NAME FROM Fires WHERE FIRE_YEAR >= 2010 AND FIRE_SIZE > 1000 AND FIPS_CODE <> "None"', engine)

In [6]:
df_us_fires.head()

,FIRE_NAME,FIRE_YEAR,FIRE_SIZE,STAT_CAUSE_DESCR,LONGITUDE,LATITUDE,FIPS_CODE,FIPS_NAME
0,WATERHOLE,2010,1818.0,Arson,-82.396389,30.321111,003,Baker
1,015 CAMP CLAIBORNE,2010,1633.0,Arson,-92.578333,31.115278,079,Rapides
2,CLAYLICK,2010,1102.0,Miscellaneous,-83.493611,38.111667,205,Rowan
3,MOTHER'S DAY,2010,3740.0,Lightning,-82.484444,30.310556,023,Columbia
4,SCHULTZ,2010,15075.0,Campfire,-111.623889,35.286389,005,Coconino


In [25]:
# Prompt for year to process.
fire_year = input("Enter year: ")


Enter year: 2015


In [26]:
df_fire_season = df_us_fires[df_us_fires['FIRE_YEAR'] == int(fire_year)]
df_fire_season.sort_values(by=["FIRE_SIZE"], ascending=False, inplace=True)
#df_fire_season.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [27]:
df_fire_season['text'] = df_fire_season['FIRE_NAME'] + '<br>Fire Size ' + (df_fire_season['FIRE_SIZE']).astype(str)+' acres'
limits = [(0,2),(3,10),(11,20),(21,50),(50,8000)]
colors = ["rgb(204,0,0)","rgb(255,0,0)","rgb(255,128,0)","rgb(255,153,153)","rgb(255,229,204)"]
states = []
scale = 500

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [28]:
for i in range(len(limits)):
    lim = limits[i]
    df_subset = df_fire_season[lim[0]:lim[1]]
    state = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_subset['LONGITUDE'],
        lat = df_subset['LATITUDE'],
        text = df_subset['text'],
        marker = dict(
            size = df_subset['FIRE_SIZE']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    states.append(state)

In [29]:
layout = dict(
        title = str(fire_year) + ' Wildfires',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ), 
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=states, layout=layout )
py.plotly.iplot( fig, validate=False, filename='d3-bubble-map-wildfire' )

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~kashs/0 or inside your plot.ly account where it is named 'd3-bubble-map-wildfire'


In [19]:
engine.dispose()